### Packets

In [163]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [164]:
import os
os.getcwd()

'c:\\Users\\Usuario\\OneDrive\\Documentos\\UPV\\Servipoli\\Inst_Ing_Energ\\Entorno\\Chome'

## Utilities

### Bot Initialize

In [165]:
link_YTstudio = "https://studio.youtube.com/channel/UCkanTv6l_aY6PM_lskgbrlg?c=UCkanTv6l_aY6PM_lskgbrlg"

def create_bot():
    chrome_path = "C:\\Program Files\\Google\\Chrome Beta\\Application\\chrome.exe"
    path_UserData = "C:\\Users\\Usuario\\AppData\\Local\\Google\\Chrome Beta\\User Data\\"
    # chrome_path = "..\\Entorno\\Chome\\GoogleChromePortableBeta\\App\\Chrome-bin\\chrome.exe"
    # path_UserData = "..\\Entorno\\Chome\\GoogleChromePortableBeta\\App\\DefaultData\\"
    driver_path = "../Structurer/chromedriver.exe"

    options = webdriver.ChromeOptions()
    # options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.add_argument("--log-level=3")
    options.add_argument("user-data-dir=" + path_UserData)
    options.add_argument("start-maximized")
    # options.add_argument('--headless')
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-extensions")
    # options.add_argument("--disable-gpu")
    # options.add_argument("--remote-debugging-port=9222")
    options.binary_location = chrome_path

    return webdriver.Chrome(options, Service(driver_path))

bot = create_bot()

### Iterator

In [166]:
def iter_videos(name: str, func: type(create_bot), *atr):
    bot.get(f"https://studio.youtube.com/channel/UCkanTv6l_aY6PM_lskgbrlg/videos/upload?c=UCkanTv6l_aY6PM_lskgbrlg&filter=%5B%7B%22name%22%3A%22TITLE%22%2C%22value%22%3A%7B%22name%22%3A%22CONTAINS%22%2C%22value%22%3A%22{name}%22%7D%7D%5D&sort=%7B%22columnType%22%3A%22date%22%2C%22sortOrder%22%3A%22ASCENDING%22%7D")

    time.sleep(1)

    videos = bot.find_elements(By.CSS_SELECTOR, "div[role=table] ytcp-video-row")

    for i in range(len(videos)):
        title = videos[i].find_element(By.CSS_SELECTOR, "a#video-title")
        text = title.text
        title.click()
        time.sleep(1)

        print(f"video {i}: {text}")

        try:
            func(*atr)
            bot.find_element(By.CSS_SELECTOR, "ytcp-button#save").click()
            print("save")
        except:
            bot.find_element(By.CSS_SELECTOR, "ytcp-button#discard").click()
            print("discard")


        bot.find_element(By.CSS_SELECTOR, "div#contentIcon").click()
        time.sleep(1)
        videos = bot.find_elements(By.CSS_SELECTOR, "div[role=table] ytcp-video-row")

In [167]:
def possibleAlert(ret: bool):
    try:
        alert = bot.switch_to.alert
        if ret:
            alert.accept()
        else:
            alert.dismiss()
    except:
        pass

In [168]:
def go_video(code: str, func: type(create_bot), *atr):
    bot.get("https://studio.youtube.com/channel/UCkanTv6l_aY6PM_lskgbrlg?c=UCkanTv6l_aY6PM_lskgbrlg")

    time.sleep(1)

    try:
        bot.find_element(By.CSS_SELECTOR, "ytcp-icon-button#search-icon").click()
    except:
        pass

    search = bot.find_element(By.CSS_SELECTOR, "input#query-input")

    search.clear()
    time.sleep(1)
    search.send_keys(code)

    time.sleep(2)

    title = bot.find_element(By.CSS_SELECTOR, "a#video-title")
    text = title.text
    title.click()
    time.sleep(1)

    print(f"video {code}: {text}")

    try:
        func(*atr)
        time.sleep(1)
        bot.find_element(By.CSS_SELECTOR, "ytcp-button#save").click()
        print("save")
    except Exception as ex:
        print(f"Error: {ex.__name__}")
        bot.find_element(By.CSS_SELECTOR, "ytcp-button#discard").click()
        print("discard")

    
    time.sleep(1)
    possibleAlert(True)
    bot.find_element(By.CSS_SELECTOR, "div#contentIcon").click()
    time.sleep(1)
    videos = bot.find_elements(By.CSS_SELECTOR, "div[role=table] ytcp-video-row")


In [169]:
go_video("TWfPtDfmYMc", lambda: print("I'm in"))

video TWfPtDfmYMc: MC01.1.1.1 - Understanding Environmental Worldviews
I'm in
save


### List selector

In [170]:
# bot.find_element(By.CSS_SELECTOR, "tp-yt-iron-icon#right-icon").click()

In [171]:
# list_selector = bot.find_element(By.CSS_SELECTOR, "array-selector#selector")

In [172]:
# list_selector.get_property("__data")["items"]

In [173]:
def list_in(*name: tuple[str]):
    bot.find_element(By.CSS_SELECTOR, "tp-yt-iron-icon#right-icon").click()
    time.sleep(1)
    
    lists = bot.find_elements(By.CSS_SELECTOR, "div#items ytcp-ve")
    list_selector = bot.find_element(By.CSS_SELECTOR, "array-selector#selector")
    info = list_selector.get_property("__data")["items"]

    for i in range(len(info)):
        if any(filter(lambda x: info[i]["label"] == x, name)) != info[i]["checked"]:
            lists[i].find_element(By.CSS_SELECTOR, "div#checkbox-container").click()

    bot.find_element(By.CSS_SELECTOR, "ytcp-button.done-button div").click()
    # list_selector.get_property("__data")["items"][0]["checked"]
    # lists[0].find_element(By.CSS_SELECTOR, "div#checkbox").click()

In [174]:
# iter_videos("MC24", list_in, "MC24")

## MC information

### functions

In [175]:
import os
import re
import docx
from fuzzywuzzy import fuzz

In [176]:
def similar(pattern: str, place: list):
    return max([(fuzz.token_set_ratio(pattern,i),i) for i in place])[1]

similar("hola", ["caracola", "alho", "hello"])

'caracola'

In [177]:
def word_find(title : str, partner : str):
    doc_path = "../MC proposals from partners/"
    
    dir1 = os.listdir(doc_path)
    
    Inst = list(filter(lambda x: re.match(".*"+partner+".*", x), dir1))[0]
    
    dir2 = os.listdir(doc_path + Inst)
    
    try:
        doc = list(filter(lambda x: re.match(".*"+title+".*", x), dir2))[0]
    except IndexError:
        # doc = similar((f"{partner} - {title}.docx"), list(map(lambda x: x, dir2)))
        doc = similar((f"{partner} - {title}.docx")[::-1], list(map(lambda x: x[::-1], dir2)))[::-1]
    
    return doc_path + Inst + "/" + doc

### Input

In [178]:
MC_code = "MC44"
doc_word = ""

### Description

In [179]:
_ = open("../Data/Docs.csv")
iterator = [i.replace("\n", "").split("\t") for i in _.readlines()]
for row in iterator:
    if str(row[0]) == MC_code:
        MC_name = str(row[1])
        if doc_word == "":
            doc_word = word_find(row[1], row[2])
_.close()
del _

print(doc_word)

../MC proposals from partners/12 - HU/HU08 Sustainable business models.docx


In [180]:
doc = docx.Document(doc_word)

Intro = doc.tables[0]
Description = doc.tables[1]

Intro_dic = {
    Intro.cell(0,0).text[:-1]: Intro.cell(0,1).text,
    Intro.cell(1,0).text[:-1]: Intro.cell(1,1).text,
    Intro.cell(2,0).text[:-1]: Intro.cell(2,1).text
}

Description_dic = {
    Description.cell(0,0).text: Description.cell(0,1).text,
    Description.cell(1,0).text: Description.cell(1,1).text,
    
    Description.cell(2,0).text: Description.cell(2,1).text,
    Description.cell(2,2).text: Description.cell(2,3).text,
    Description.cell(2,4).text: Description.cell(2,5).text,
    
    Description.cell(3,0).text: Description.cell(3,1).text,
    
    Description.cell(4,0).text:  (Description.cell( 5,0).text),
    Description.cell(6,0).text:  (Description.cell( 7,0).text),
    Description.cell(8,0).text:  (Description.cell( 9,0).text),
    Description.cell(10,0).text: (Description.cell(11,0).text),
    Description.cell(12,0).text: (Description.cell(13,0).text),
    Description.cell(14,0).text: (Description.cell(15,0).text),
    Description.cell(16,0).text: (Description.cell(17,0).text)
}

print(Intro_dic)

{'Institution': 'Halmstad University', 'Module coordinator': 'Pia Ulvenblad', 'Lecturers': 'Pia Ulvenblad'}


In [181]:
print(Description_dic.keys())

dict_keys(['MC title', 'MC long title', 'ISCED codes', 'EQF level,', 'ECTS', 'Suitable for', 'Background of the proposed micro-credential', 'Overview of the micro-credential', 'Learning objectives', 'Table of contents', 'Teaching and learning methods', 'Prerequisites', 'Assessment methods'])


In [182]:
Description_dic['Overview of the micro-credential']

'The course focuses on knowledge about how organizations can work strategically and sustainably with issues related to business models and development of value. Included in the business models are the following parts: (i) value proposition, (ii) value creation and delivery, (iii) value capture and (iv) value intention.'

In [183]:
# (MC_code, MC_name, Overview, Lecturers)
desc_plantilla = """This video belongs to a Master of RES4City in https://www.res4city.eu/

Unit: {0}

Course: {1}

{2}

{3}


Lecturers: {4}

{5}"""

In [184]:
import openpyxl

MC_path = f"../MC_input/{MC_code}/" 

WB_in_name = [i for i in os.listdir(MC_path) if re.match(f".*{MC_code}.*xls.+$", i)][0]
print(WB_in_name)
WB_in = openpyxl.load_workbook(filename= MC_path + WB_in_name, read_only= True, keep_vba= True, keep_links= True)

MC_Template = WB_in.get_sheet_by_name("Template")

MC_name = MC_Template.cell(2,3).value

MC_lecturer = MC_Template.cell(3,3).value

MC_inst = MC_Template.cell(4,3).value

MC44 RES4CITY - NEW final Ulvenblad Sustainable Business Models new adjusted version.xlsx


C:\Users\Usuario\AppData\Local\Temp\ipykernel_14048\4184641218.py:9: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  MC_Template = WB_in.get_sheet_by_name("Template")


## Start

In [185]:
def ins_desc(n_sec, n_sub, n_unit, title) -> None:
    time.sleep(1)

    title_text = ".".join([str(i) for i in [MC_code, n_sec, n_sub, n_unit]]) + " - " + title
    desc_text = desc_plantilla.format(
        f"S{n_sec}-Sub{n_sub}-U{n_unit}",
        MC_code, 
        MC_name, 
        Description_dic['Overview of the micro-credential'],
        MC_inst,
        Intro_dic['Lecturers'])

    title_input, description_input = bot.find_elements(By.XPATH, '//*[@id="textbox"]')
    
    if title_input.text != title_text:

        while title_input.text != "":
            title_input.clear()
            time.sleep(1)
        
        title_input.send_keys(title_text)
        time.sleep(1)
    
    if description_input.text != desc_text.strip():

        while description_input.text != "":
            description_input.clear()
            time.sleep(1)
        
        description_input.send_keys(desc_text)
        time.sleep(1)

In [186]:
videos = []

In [187]:
print(videos)

[]


In [188]:
# videos.append("JRci2V8PxW4")

In [189]:
sec_n, sub_n = (1,) * 2
sec, sub = ("",) * 2

def unit(row: tuple, n_sec: int, n_sub: int):
    
    n_unit = row[1].value
    title = row[2].value
    
    video = row[3].value

    def func():
        ins_desc(n_sec, n_sub, n_unit, title)
        list_in(MC_code, MC_code + " - " + MC_name)

    if type(video) != str:
        return
    
    if len(video) == 11:
        if video not in videos:
            go_video(video, func)
            videos.append(video)

for row in MC_Template.iter_rows():

    header = str(row[0].value)

    if bool(re.match("^Section.*[0-9]+.*$",header)):
        sec_n = int(re.findall("[0-9]+", header)[0])
        sec = row[1].value
        sub = row[1].value
        sub_n = 1
        continue
    elif bool(re.match("^Subsection.*$",header)):
        sub_n = int(row[1].value)
        sub = row[2].value
        continue
    elif bool(re.match("^.*Unit.*$",header)):
        unit(row, sec_n, sub_n)

print("DONE")

video JT5hGtY9Afk: MC44 - Introduction
save
video Cf-RHsk2Oe0: MC44 - The sustainability concept
save
video SzqXLshRdls: MC44 - Global sustainability goals
save
video QbGdv7on2jY: MC44 - Sustainability goals in focus
save
video 2ebdC93ueDw: MC44 - To meet the global challenges
save
video jmQv0G9K5eg: MC44 - Parts in a business model
save
video UaiwGUjwB8c: MC44 - Business model for sustainability
save
video 31DYmp2pTlw: MC44 - The entrepreneur and stewardship
save
video weJYYjaMKDM: MC44 - Circular business models
save
video DKVMWkOYPbU: MC44 - Business model canvas, part 1
save
video xR2W-tuMeFE: MC44 - Business model canvas, part 2
save
video y9tWYL_P-oI: MC44 - Sustainability and circularity
save
video 5b9aeu9va8w: MC44 - Barriers in the development
save
video WmuH359OWvo: MC44 - Summary
save
DONE


## Finish

In [190]:
playlist = False

In [191]:
time.sleep(2)

bot.get(f"https://studio.youtube.com/channel/UCkanTv6l_aY6PM_lskgbrlg/content/playlists?c=UCkanTv6l_aY6PM_lskgbrlg")

time.sleep(1)

bot.refresh()
time.sleep(2)

# bot.find_element(By.CSS_SELECTOR, "#footer ytcp-dropdown-trigger").click()

# bot.find_element(By.CSS_SELECTOR, "#text-item-2").click()

# time.sleep(1)

videos = bot.find_elements(By.CSS_SELECTOR, "div#playlist-table-content ytcp-playlist-row")

print(videos)

for i in range(len(videos)):
    title = videos[i].find_element(By.CSS_SELECTOR, "a#playlist-title-link")


    if (title.text).strip() != MC_code and (title.text).strip() != (MC_code + " - " + MC_name):
        continue

    title.click()
    time.sleep(1)
    title_input, description_input = bot.find_elements(By.XPATH, '//*[@id="textbox"]')

    while title_input.text != "":
        title_input.clear()
        time.sleep(1)
    
    title_input.send_keys(MC_code + " - " + MC_name)

    while description_input.text != "":
        description_input.clear()
        time.sleep(1)
    
    description_input.send_keys(
        "These are the videos of a Master of RES4City.\n\nFor more information, please visit the following website:\nhttps://www.res4city.eu/"
    )

    try:
        bot.find_element(By.CSS_SELECTOR, "ytcp-button#save").click()
        print("save")
    except Exception as ex:
        print(f"Error: {ex.__name__}")
        bot.find_element(By.CSS_SELECTOR, "ytcp-button#discard").click()
        print("discard")
    
    time.sleep(1)
    possibleAlert(True)
    bot.find_element(By.CSS_SELECTOR, "div#contentIcon").click()

    videos = bot.find_elements(By.CSS_SELECTOR, "div#playlist-table-content ytcp-playlist-row")

[<selenium.webdriver.remote.webelement.WebElement (session="93591fa28bb552f773bdde5feae041b8", element="B26452B87E1997105DF513B264D967A7_element_1852")>, <selenium.webdriver.remote.webelement.WebElement (session="93591fa28bb552f773bdde5feae041b8", element="B26452B87E1997105DF513B264D967A7_element_1853")>, <selenium.webdriver.remote.webelement.WebElement (session="93591fa28bb552f773bdde5feae041b8", element="B26452B87E1997105DF513B264D967A7_element_1854")>, <selenium.webdriver.remote.webelement.WebElement (session="93591fa28bb552f773bdde5feae041b8", element="B26452B87E1997105DF513B264D967A7_element_1855")>, <selenium.webdriver.remote.webelement.WebElement (session="93591fa28bb552f773bdde5feae041b8", element="B26452B87E1997105DF513B264D967A7_element_1856")>, <selenium.webdriver.remote.webelement.WebElement (session="93591fa28bb552f773bdde5feae041b8", element="B26452B87E1997105DF513B264D967A7_element_1857")>, <selenium.webdriver.remote.webelement.WebElement (session="93591fa28bb552f773bdde

In [192]:
bot.get(f"https://studio.youtube.com/channel/UCkanTv6l_aY6PM_lskgbrlg/content/playlists?c=UCkanTv6l_aY6PM_lskgbrlg")

time.sleep(1)

videos = bot.find_elements(By.CSS_SELECTOR, "div#playlist-table-content ytcp-playlist-row")

for i in range(len(videos)):
    title = videos[i].find_element(By.CSS_SELECTOR, "a#playlist-title-link")

    if (title.text).strip() != (MC_code + " - " + MC_name):
        playlist = True

assert playlist

In [193]:
bot.close()